In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
import torch
import re

# 테스트 데이터셋 로드 (KLUE STS)
dataset = load_dataset('klue', 'sts')

print(dataset[50])

OlmoForCausalLM(
  (model): OlmoModel(
    (embed_tokens): Embedding(50304, 4096, padding_idx=1)
    (layers): ModuleList(
      (0-31): 32 x OlmoDecoderLayer(
        (self_attn): OlmoAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (v_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): OlmoMLP(
          (gate_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (up_proj): Linear(in_features=4096, out_features=11008, bias=False)
          (down_proj): Linear(in_features=11008, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): OlmoLayerNorm()
        (post_attention_layernorm): OlmoLayerNorm()
      )
    )
    (norm): OlmoLayerNorm()
    (rotary_emb): OlmoRotaryEmbedding()
  )
  (lm_head

In [12]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
import torch
import re
from sklearn.metrics import mean_squared_error, mean_absolute_error
import time
import os
import json
import pandas as pd
import requests

# Ollama 관련 설정 및 함수
try:
    import ollama
    OLLAMA_AVAILABLE = True
    # Ollama 서버 설정
    ollama_host = "http://sg016:11434"
    ollama_client = ollama.Client(host=ollama_host)
except ImportError:
    OLLAMA_AVAILABLE = False
    print("Ollama 패키지가 설치되어 있지 않습니다. pip install ollama로 설치하세요.")

def check_ollama_server():
    """Ollama 서버 연결 상태 확인"""
    if not OLLAMA_AVAILABLE:
        return False
        
    try:
        response = requests.get(f"{ollama_host}/api/version")
        print(f"Ollama 서버 연결됨: 버전 {response.json().get('version', '알 수 없음')}")
        return True
    except Exception as e:
        print(f"Ollama 서버 연결 실패: {e}")
        return False

# Hugging Face 모델 관련 함수
def load_model_and_tokenizer(model_path, use_local=True, dtype=torch.bfloat16):
    """모델과 토크나이저를 로드하는 함수"""
    print(f"모델 로딩 중: {model_path}")
    try:
        tokenizer = AutoTokenizer.from_pretrained(
            model_path, 
            local_files_only=use_local
        )
        
        model = AutoModelForCausalLM.from_pretrained(
            model_path,
            local_files_only=use_local,
            torch_dtype=dtype,
            device_map="auto"
        )
        print("모델 로딩 완료!")
        return model, tokenizer
    except Exception as e:
        print(f"모델 로딩 중 오류 발생: {e}")
        return None, None

def predict_similarity_with_model(model, tokenizer, sentence1, sentence2):
    """특정 모델을 사용하여 두 문장의 유사도를 예측하는 함수"""
    prompt = f"Analyze the following sentence pair and provide a similarity score between 0 and 5, where 0 means no similarity and 5 means identical.\n\nSentence 1: {sentence1}\nSentence 2: {sentence2}\n\nSimilarity Score:"
    
    # 토크나이즈
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    # 추론 수행
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=15,
            temperature=0.1,
            top_p=0.95,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    # 결과 디코딩
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # 프롬프트 이후의 생성된 텍스트만 추출
    response = response[len(prompt):]
    
    # 숫자 추출 시도
    try:
        # 0~5 사이의 숫자 찾기
        match = re.search(r'([0-5](\.\d+)?)', response)
        if match:
            predicted_score = float(match.group(1))
        else:
            # 숫자를 찾지 못한 경우 기본값 설정
            print(f"점수 추출 불가: '{response}'. 기본값 사용.")
            predicted_score = 2.5  # 기본값
    except Exception as e:
        print(f"점수 추출 오류: {e}. 응답: '{response}'")
        predicted_score = 2.5  # 오류 발생 시 기본값
    
    # 0~5 범위로 제한
    predicted_score = max(0, min(5, predicted_score))
    
    return predicted_score

def predict_similarity_ollama(sentence1, sentence2, model_name="llama3.2", language="en"):
    """Ollama API를 사용하여 두 문장의 유사도를 예측하는 함수"""
    if not OLLAMA_AVAILABLE:
        print("Ollama 패키지를 사용할 수 없습니다.")
        return 2.5
 
    prompt = f"Analyze the following sentence pair and provide a similarity score between 0 and 5, where 0 means no similarity and 5 means identical.\n\nSentence 1: {sentence1}\nSentence 2: {sentence2}\n\nReply with only a number."
    
    try:
        # 문장 쌍을 프롬프트와 함께 모델에 입력
        response = ollama_client.chat(
            model=model_name, 
            messages=[{
                'role': 'user',
                'content': prompt,
            }],
            options={
                'temperature': 0.1,  # 낮은 온도로 일관된 결과
                'top_p': 0.95,
            }
        )
        
        # 응답에서 점수 추출
        content = response['message']['content'].strip()
        
        # 정규 표현식으로 숫자 추출
        match = re.search(r'([0-5](\.\d+)?)', content)
        if match:
            predicted_score = float(match.group(1))
        else:
            # 숫자를 찾지 못한 경우 2.5로 기본 설정
            print(f"응답에서 점수를 추출할 수 없습니다: '{content}'. 기본값 사용.")
            predicted_score = 2.5
            
        # 0~5 범위로 제한
        predicted_score = max(0, min(5, predicted_score))
        
        return predicted_score
        
    except Exception as e:
        print(f"예측 중 오류 발생: {e}")
        return 2.5  # 오류 발생 시 기본값

# 통합 평가 함수
def evaluate_model_on_klue_sts(model_path, eval_samples=100, save_results=True, output_dir="results"):
    """KLUE STS 데이터셋을 사용하여 모델(HF 또는 Ollama)을 평가하는 함수"""
    # 모델 정보 추출
    model_type = model_info.get("type", "hf")  # 기본값은 huggingface
    model_name = model_info.get("name", "Unknown")
    language = model_info.get("language", "en")
    
    if model_type == "ollama":
        # Ollama 서버 확인
        if not check_ollama_server():
            print("Ollama 서버에 연결할 수 없어 평가를 건너뜁니다.")
            return None
        model_path = model_info.get("path", model_name)  # Ollama의 경우 path는 모델명과 동일
    else:  # huggingface
        model_path = model_info.get("path")
        use_local = model_info.get("local", True)
        
        # 모델 및 토크나이저 로드 (HF 모델만)
        model, tokenizer = load_model_and_tokenizer(model_path, use_local)
        if model is None or tokenizer is None:
            return None
        
        # 평가 모드로 전환
        model.eval()
    
    # 데이터셋 로드
    print("KLUE STS 데이터셋 로드 중...")
    dataset = load_dataset('klue', 'sts')
    
    # 예측 및 레이블 저장용 리스트
    predictions = []
    labels = []
    examples_data = []  # 디버깅 및 분석용 데이터 저장
    
    # 시작 시간 기록
    start_time = time.time()
    
    # 평가할 샘플 수 설정
    if eval_samples is None or eval_samples > len(dataset['validation']):
        eval_samples = len(dataset['validation'])
    
    print(f"총 {eval_samples}개 샘플 평가 시작...")
    for i, example in enumerate(dataset['validation']):
        if i >= eval_samples:
            break
            
        sentence1 = example['sentence1']
        sentence2 = example['sentence2']
        label = example['labels']['label']  # 실제 유사도 점수
        
        # 두 문장에 대해 유사도 예측 (모델 타입에 따라 다른 함수 사용)
        if model_type == "ollama":
            pred = predict_similarity_ollama(sentence1, sentence2, model_info["path"])
        else:  # huggingface
            pred = predict_similarity_with_model(model, tokenizer, sentence1, sentence2)
            
        predictions.append(pred)
        labels.append(label)
        
        # 예제 데이터 저장
        examples_data.append({
            'sentence1': sentence1,
            'sentence2': sentence2,
            'true_score': label,
            'predicted_score': pred
        })
        
        # 진행 상황 출력
        if (i+1) % 10 == 0:
            elapsed = time.time() - start_time
            avg_time = elapsed / (i+1)
            remaining = avg_time * (eval_samples - (i+1))
            print(f"처리됨: {i+1}/{eval_samples} 예제 | 평균 시간: {avg_time:.2f}초/예제 | 남은 시간: {remaining/60:.2f}분")
    
    # 평가 지표 계산
    mse = mean_squared_error(labels, predictions)
    mae = mean_absolute_error(labels, predictions)
    
    # 결과 출력
    print("\n평가 결과:")
    print(f"모델: {model_name}")
    print(f"Mean Squared Error (MSE): {mse:.4f}")
    print(f"Mean Absolute Error (MAE): {mae:.4f}")
    
    # 결과 저장
    if save_results:
        # 결과 디렉토리 생성
        os.makedirs(output_dir, exist_ok=True)
        
        # 파일명에 사용할 안전한 모델명 생성
        safe_model_name = model_name.replace("/", "-")
        
        # 결과 요약 저장
        results = {
            'model_name': model_name,
            'model_path': model_path,
            'model_type': model_type,
            'eval_samples': eval_samples,
            'mse': float(mse),
            'mae': float(mae),
            'timestamp': time.strftime("%Y-%m-%d %H:%M:%S")
        }
        
        with open(f"{output_dir}/{safe_model_name}_summary.json", 'w', encoding='utf-8') as f:
            json.dump(results, f, ensure_ascii=False, indent=2)
        
        # 세부적인 예제별 결과 저장
        df = pd.DataFrame(examples_data)
        df.to_csv(f"{output_dir}/{safe_model_name}_detailed.csv", index=False, encoding='utf-8')
        
        print(f"결과가 {output_dir} 디렉토리에 저장되었습니다.")
    
    return {"mse": mse, "mae": mae}

In [ ]:
# 평가할 모델 경로 목록
models_to_evaluate = [
    {
        "type": "hf",
        "path": "allenai/OLMo-7B-hf",
        "name": "OLMo-7B",
    },
    {
        "type": "hf",
        "path": "fine-tuned-models/fine-tuned-olmo7B-v12-80000",
        "name": "OLMo-7B-Fine-Tuned",
    },
    {
        "type": "hf",
        "path": "allenai/OLMo-1B-hf",
        "name": "OLMo-1B",
    },
    {
        "type": "hf",
        "path": "fine-tuned-models/fine-tuned-olmo1B-80000-v11",
        "name": "OLMo-1B-Fine-Tuned",
    },
    {
        "type": "ollama",
        "path": "llama3.2",
        "name": "Llama-3.2",
    }
]

# 결과 저장을 위한 디렉토리
results_dir = "klue_sts_results"

# 각 모델 평가
all_results = {}
for model_info in models_to_evaluate:
    print(f"\n===== {model_info['name']} 평가 시작 =====")
    results = evaluate_model_on_klue_sts(
        model_path=model_info["path"],
        eval_samples=len(dataset['validation']),  # 전체 검증 세트 사용
        save_results=True,
        output_dir=results_dir
    )
    if results:
        all_results[model_info["name"]] = results

# 모든 모델 결과 비교
if len(all_results) > 1:
    print("\n===== 모델 성능 비교 =====")
    for model_name, metrics in all_results.items():
        print(f"{model_name}: MSE={metrics['mse']:.4f}, MAE={metrics['mae']:.4f}")



===== OLMo-7B 평가 시작 =====
모델 로딩 중: allenai/OLMo-7B-hf


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

모델 로딩 완료!
KLUE STS 데이터셋 로드 중...
총 519개 샘플 평가 시작...
처리됨: 10/519 예제 | 평균 시간: 0.50초/예제 | 남은 시간: 4.28분
처리됨: 20/519 예제 | 평균 시간: 0.49초/예제 | 남은 시간: 4.07분
처리됨: 30/519 예제 | 평균 시간: 0.49초/예제 | 남은 시간: 3.96분
처리됨: 40/519 예제 | 평균 시간: 0.48초/예제 | 남은 시간: 3.87분
처리됨: 50/519 예제 | 평균 시간: 0.48초/예제 | 남은 시간: 3.78분
처리됨: 60/519 예제 | 평균 시간: 0.48초/예제 | 남은 시간: 3.70분
처리됨: 70/519 예제 | 평균 시간: 0.48초/예제 | 남은 시간: 3.61분
처리됨: 80/519 예제 | 평균 시간: 0.48초/예제 | 남은 시간: 3.53분
처리됨: 90/519 예제 | 평균 시간: 0.48초/예제 | 남은 시간: 3.43분
처리됨: 100/519 예제 | 평균 시간: 0.48초/예제 | 남은 시간: 3.35분
처리됨: 110/519 예제 | 평균 시간: 0.48초/예제 | 남은 시간: 3.27분
처리됨: 120/519 예제 | 평균 시간: 0.48초/예제 | 남은 시간: 3.19분
처리됨: 130/519 예제 | 평균 시간: 0.48초/예제 | 남은 시간: 3.11분
처리됨: 140/519 예제 | 평균 시간: 0.48초/예제 | 남은 시간: 3.02분
처리됨: 150/519 예제 | 평균 시간: 0.48초/예제 | 남은 시간: 2.94분
처리됨: 160/519 예제 | 평균 시간: 0.48초/예제 | 남은 시간: 2.86분
처리됨: 170/519 예제 | 평균 시간: 0.48초/예제 | 남은 시간: 2.79분
처리됨: 180/519 예제 | 평균 시간: 0.48초/예제 | 남은 시간: 2.72분
처리됨: 190/519 예제 | 평균 시간: 0.48초/예제 | 남은 시간: 2.64분
처리됨: 200/519 예제 | 평균 시간: 0.

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
import torch
import re

# 모델 경로 설정
model_path = "allenai/OLMo-7B-hf"

# 토크나이저와 모델 로드
tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)
model = AutoModelForCausalLM.from_pretrained(model_path, local_files_only=True, torch_dtype=torch.bfloat16, device_map="auto")


def predict_similarity(sentence1, sentence2):
    # 프롬프트 정의 (두 문장의 유사도 점수 평가 요청)
    prompt = f"Analyze the following sentence pair and provide a similarity score between 0 and 5, where 0 means no similarity and 5 means identical.\n\nSentence 1: {sentence1}\nSentence 2: {sentence2}\n\nSimilarity Score:"
    
    # 토크나이즈
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    
    # 추론 수행
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=10,
            temperature=0.1,
            top_p=0.95,
            do_sample=True,
            pad_token_id=tokenizer.eos_token_id
        )
    
    # 결과 디코딩
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # 프롬프트 이후의 생성된 텍스트만 추출
    response = response[len(prompt):]
    
    # 숫자 추출 시도
    try:
        # 0~5 사이의 숫자 찾기
        match = re.search(r'([0-5](\.\d+)?)', response)
        if match:
            predicted_score = float(match.group(1))
        else:
            # 숫자를 찾지 못한 경우 기본값 설정
            print(f"Unable to extract score from: '{response}'. Using default value.")
            predicted_score = 2.5  # 기본값
    except Exception as e:
        print(f"Error extracting score: {e}. Response: '{response}'")
        predicted_score = 2.5  # 오류 발생 시 기본값
    
    # 0~5 범위로 제한
    predicted_score = max(0, min(5, predicted_score))
    
    return predicted_score

# 'validation' 데이터셋에서 예측 수행
predictions = []
labels = []

# 평가 샘플 수 제한 (전체 데이터셋을 테스트하기에는 시간이 오래 걸릴 수 있음)
eval_samples = 100  # 필요에 따라 조정

for i, example in enumerate(dataset['validation']):
    if i >= eval_samples:
        break
        
    sentence1 = example['sentence1']
    sentence2 = example['sentence2']
    label = example['labels']['label']  # 실제 유사도 점수
    
    # 두 문장에 대해 유사도 예측
    pred = predict_similarity(sentence1, sentence2)
    predictions.append(pred)
    labels.append(label)
    
    # 진행 상황 출력
    if (i+1) % 10 == 0:
        print(f"Processed {i+1}/{eval_samples} examples")

# 평가 지표 계산 (MSE, MAE)
from sklearn.metrics import mean_squared_error, mean_absolute_error
mse = mean_squared_error(labels, predictions)
mae = mean_absolute_error(labels, predictions)

print("\nEvaluation Results:")
print("KLUE-STS | OLMo-7b")
print(f"Mean Squared Error (MSE): {mse:.4f}")
print(f"Mean Absolute Error (MAE): {mae:.4f}")

Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

Processed 10/100 examples
Processed 20/100 examples
Processed 30/100 examples
Processed 40/100 examples
Processed 50/100 examples
Processed 60/100 examples
Processed 70/100 examples
Processed 80/100 examples
Processed 90/100 examples
Processed 100/100 examples

Evaluation Results:
KLUE-STS | OLMo-7b
Mean Squared Error (MSE): 3.9518
Mean Absolute Error (MAE): 1.6060


In [23]:
#####################################
# KLUE STS, OLMo-7b FineTuned
#####################################

# 모델 경로 설정
model_path = 'fine-tuned-models/fine-tuned-olmo7B-v12-80000'

# 토크나이저와 모델 로드
tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)
model = AutoModelForCausalLM.from_pretrained(model_path, local_files_only=True, torch_dtype=torch.bfloat16, device_map="auto")

# 테스트 데이터셋 로드 (KLUE STS 예시, 필요에 따라 다른 데이터셋 사용)
dataset = load_dataset('klue', 'sts')

# 모델을 평가 모드로 전환
model.eval()
# 프롬프트 정의 (두 문장의 유사도 점수 평가 요청)
prompt = "Analyze the following sentence pair and provide a similarity score between 0 and 5, where 0 means no similarity and 5 means identical."

def predict_similarity(sentence1, sentence2):
    # 문장 쌍을 프롬프트와 함께 모델에 입력
    response = client.chat(
        model='llama3.2', 
        messages=[{
            'role': 'user',
            'content': f"{prompt} Sentence 1: {sentence1} Sentence 2: {sentence2}",
        }]
    )
    
    # 예측된 유사도 점수 추출 (응답에서 점수를 얻음)
    predicted_score = float(response['message']['content'].strip())
    
    return predicted_score

# 'validation' 데이터셋에서 예측 수행
predictions = []
labels = []

for example in dataset['validation']:
    sentence1 = example['sentence1']
    sentence2 = example['sentence2']
    label = example['labels']['label']  # 'labels'에서 실제 값 가져오기
    
    # 두 문장에 대해 유사도 예측
    pred = predict_similarity(sentence1, sentence2)
    predictions.append(pred)
    labels.append(label)

# 평가 지표 계산 (MSE, MAE)
from sklearn.metrics import mean_squared_error, mean_absolute_error

mse = mean_squared_error(labels, predictions)
mae = mean_absolute_error(labels, predictions)

print("Evaluation")
print("KLUE-STS | OLMo-7b fine tuned")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

Evaluation
KLUE-STS | OLMo-7b fine tuned
Mean Squared Error (MSE): 7.81635838150289
Mean Absolute Error (MAE): 2.3836223506743734


In [24]:
#####################################
# KLUE STS, OLMo-1b Original
#####################################

# 모델 경로 설정
model_path = "allenai/OLMo-1B-hf"

# 토크나이저와 모델 로드
tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)
model = AutoModelForCausalLM.from_pretrained(model_path, local_files_only=True, torch_dtype=torch.bfloat16, device_map="auto")

# 테스트 데이터셋 로드 (KLUE STS 예시, 필요에 따라 다른 데이터셋 사용)
dataset = load_dataset('klue', 'sts')

# 모델을 평가 모드로 전환
model.eval()

# 프롬프트 정의 (두 문장의 유사도 점수 평가 요청)
prompt = "Analyze the following sentence pair and provide a similarity score between 0 and 5, where 0 means no similarity and 5 means identical."

def predict_similarity(sentence1, sentence2):
    # 문장 쌍을 프롬프트와 함께 모델에 입력
    response = client.chat(
        model='llama3.2', 
        messages=[{
            'role': 'user',
            'content': f"{prompt} Sentence 1: {sentence1} Sentence 2: {sentence2}",
        }]
    )
    
    # 예측된 유사도 점수 추출 (응답에서 점수를 얻음)
    predicted_score = float(response['message']['content'].strip())
    
    return predicted_score

# 'validation' 데이터셋에서 예측 수행
predictions = []
labels = []

for example in dataset['validation']:
    sentence1 = example['sentence1']
    sentence2 = example['sentence2']
    label = example['labels']['label']  # 'labels'에서 실제 값 가져오기
    
    # 두 문장에 대해 유사도 예측
    pred = predict_similarity(sentence1, sentence2)
    predictions.append(pred)
    labels.append(label)

# 평가 지표 계산 (MSE, MAE)
from sklearn.metrics import mean_squared_error, mean_absolute_error

mse = mean_squared_error(labels, predictions)
mae = mean_absolute_error(labels, predictions)


print("Evaluation:")
print("KLUE-STS | OLMo-1b Original")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")


Evaluation:
KLUE-STS | OLMo-1b Original
Mean Squared Error (MSE): 8.107687861271677
Mean Absolute Error (MAE): 2.430635838150289


In [25]:
#####################################
# KLUE STS, OLMo-1b FineTuned
#####################################

# 모델 경로 설정
model_path = "./fine-tuned-models/fine-tuned-olmo1B-80000-v11"

# 토크나이저와 모델 로드
tokenizer = AutoTokenizer.from_pretrained(model_path, local_files_only=True)
model = AutoModelForCausalLM.from_pretrained(model_path, local_files_only=True, torch_dtype=torch.bfloat16, device_map="auto")

# 테스트 데이터셋 로드 (KLUE STS 예시, 필요에 따라 다른 데이터셋 사용)
dataset = load_dataset('klue', 'sts')

# 모델을 평가 모드로 전환
model.eval()
# 프롬프트 정의 (두 문장의 유사도 점수 평가 요청)
prompt = "Analyze the following sentence pair and provide a similarity score between 0 and 5, where 0 means no similarity and 5 means identical."

def predict_similarity(sentence1, sentence2):
    # 문장 쌍을 프롬프트와 함께 모델에 입력
    response = client.chat(
        model='llama3.2', 
        messages=[{
            'role': 'user',
            'content': f"{prompt} Sentence 1: {sentence1} Sentence 2: {sentence2}",
        }]
    )
    
    # 예측된 유사도 점수 추출 (응답에서 점수를 얻음)
    predicted_score = float(response['message']['content'].strip())
    
    return predicted_score

# 'validation' 데이터셋에서 예측 수행
predictions = []
labels = []

for example in dataset['validation']:
    sentence1 = example['sentence1']
    sentence2 = example['sentence2']
    label = example['labels']['label']  # 'labels'에서 실제 값 가져오기
    
    # 두 문장에 대해 유사도 예측
    pred = predict_similarity(sentence1, sentence2)
    predictions.append(pred)
    labels.append(label)

# 평가 지표 계산 (MSE, MAE)
from sklearn.metrics import mean_squared_error, mean_absolute_error

mse = mean_squared_error(labels, predictions)
mae = mean_absolute_error(labels, predictions)


print("Evaluation:")
print("KLUE-STS | OLMo-1b FineTuned")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")


Evaluation:
KLUE-STS | OLMo-1b FineTuned
Mean Squared Error (MSE): 7.588998073217726
Mean Absolute Error (MAE): 2.340462427745665


In [27]:
import requests
import ollama
from transformers import AutoTokenizer, AutoModelForCausalLM
from datasets import load_dataset
import torch
from sklearn.metrics import mean_squared_error, mean_absolute_error
import re  # 정규 표현식 모듈 추가

# Ollama 서버 설정
ollama_host = "http://sg001:11434"
client = ollama.Client(host=ollama_host)  # 클라이언트 인스턴스 생성

# 서버 연결 확인
try:
    response = requests.get(ollama_host)
    print("Server connected")
    print(response.text)
except requests.ConnectionError:
    print("Not connected")

# Llama3.2 모델을 사용하여 분석
# Llama3.2 모델을 위한 프롬프트를 설정합니다
prompt = "Analyze the following sentence pair and provide a similarity score between 0 and 5, where 0 means no similarity and 5 means identical."

# KLUE STS 데이터셋 로드
dataset = load_dataset('klue', 'sts')

# 프롬프트 정의 (두 문장의 유사도 점수 평가 요청)
prompt = "Analyze the following sentence pair and provide a similarity score between 0 and 5, where 0 means no similarity and 5 means identical."

def predict_similarity(sentence1, sentence2):
    # 문장 쌍을 프롬프트와 함께 모델에 입력
    response = client.chat(
        model='llama3.2', 
        messages=[{
            'role': 'user',
            'content': f"{prompt} Sentence 1: {sentence1} Sentence 2: {sentence2}",
        }]
    )
    
    # 예측된 유사도 점수 추출 (응답에서 점수를 얻음)
    predicted_score = float(response['message']['content'].strip())
    
    return predicted_score

# 'validation' 데이터셋에서 예측 수행
predictions = []
labels = []

for example in dataset['validation']:
    sentence1 = example['sentence1']
    sentence2 = example['sentence2']
    label = example['labels']['label']  # 'labels'에서 실제 값 가져오기
    
    # 두 문장에 대해 유사도 예측
    pred = predict_similarity(sentence1, sentence2)
    predictions.append(pred)
    labels.append(label)

# 평가 지표 계산 (MSE, MAE)
from sklearn.metrics import mean_squared_error, mean_absolute_error

mse = mean_squared_error(labels, predictions)
mae = mean_absolute_error(labels, predictions)

print("Evaluation")
print("KLUE-STS | Llama3.2")
print(f"Mean Squared Error (MSE): {mse}")
print(f"Mean Absolute Error (MAE): {mae}")


Server connected
Ollama is running
Evaluation
KLUE-STS | Llama3.2
Mean Squared Error (MSE): 3.6705973025048175
Mean Absolute Error (MAE): 1.5129094412331405
